[View in Colaboratory](https://colab.research.google.com/github/shaikhf04/Hello-World/blob/master/max.ipynb)

In [5]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

12 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2018-10-24 13:20:24--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?QfQLTPDvDH1BIhJbHFhnbBFL--mrBwY6VMgAIM--4XVXLRbZAmsh3FW-fb8c3N6QuU2ZAv8z1Qza3F5SNwnFMPWa6Nlkfg5V4Qeamks53buIE7dfjT425o9kVOr5EV9yN6dg1nP1XbwLaAtS-8bVMQSZ5BeR2L1ZnigT8uZE_Frk1Uej0tKP01CItZIx06qZR7CgMVdsFg8-vhU7hcc86IMDMA [following]
--2018-10-24 13:20:24--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?QfQLTPDvDH1BIhJbHFhnbB

In [6]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Tue_Jun_12_23:07:04_CDT_2018
Cuda compilation tools, release 9.2, V9.2.148


In [7]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qln1hp5a
  Running setup.py bdist_wheel for NVCCPlugin ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-bwxevog6/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [9]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [10]:
%%cu
#include <iostream>
int main() {
    std::cout << "Hello world\n";
    return 0;
}

'Hello world\n'

In [11]:
%%cu
#include<iostream>
#include<cstdio>

using namespace std;

__global__ void mini1(int *a,int *b,int n)
{
    int block=256*blockIdx.x;
    int mini=7888888;
    
    for(int i=block;i<min(256+block,n);i++)
    {
        if(mini>a[i])
        {
            mini=a[i];
        }
    }
    b[blockIdx.x]=mini;
}

int main()
{
    int n;
    n=100;
    int a[n];
    
    cudaEvent_t start,end;
    
    for(int i=0;i<n;i++)
    {
        a[i]=i+1;
    }
    
    int *ad,*bd;
    int size=n*sizeof(int);
    
    cudaMalloc(&ad,size);
    cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);
    
    int grids=ceil(n*1.0f/256.0f);
    cudaMalloc(&bd,grids*sizeof(int));
    
    dim3 grid(grids,1);
    dim3 block(1,1);
    
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    
    cudaEventRecord(start);
    
    while(n>1)
    {
        mini1<<<grids,block>>>(ad,bd,n);
        n=ceil(n*1.0f/256.0f);
        cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
    }
    
    cudaEventRecord(end);
    cudaEventSynchronize(end);
    
    float time = 0;
    cudaEventElapsedTime(&time,start,end);
    
    int ans[2];
    cudaMemcpy(ans,ad,4,cudaMemcpyDeviceToHost);

    cout<<"Minimum numbers are : "<<ans[0]<<endl;
    cout<<"Total time require :  "<<time<<endl;
  
}

/*

'Minimum numbers are : 1\nTotal time require :  0.099808\n'

*/


'Minimum numbers are : 1\nTotal time require :  0.141248\n'

In [14]:
%%cu
#include<iostream>
#include<cstdio>

using namespace std;



__global__ void mini1(int *a,int *b,int n)
{

	int block=256*blockIdx.x;
	int mini=7888888;

	for(int i=block;i<min(256+block,n);i++)
	{
		if(mini>a[i])
		{
		mini=a[i];
	
		}
	}
	b[blockIdx.x]=mini;

}




int main()
{


	int n=5;
	cin>>n;
	int a[n];

	cudaEvent_t start,end;

	cudaEventCreate(&start);
	cudaEventCreate(&end);

	for(int i=0;i<n;i++)
	{

		a[i]=i+1;
	}

	
	int *ad,*bd;
	int size=n*sizeof(int);
	
	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	

	int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));



	dim3 grid(grids,1);
	dim3 block(1,1);

	
	cudaEventRecord(start);

	

	while(n>1)

	{

		mini1<<<grids,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);


	}

	
	cudaEventRecord(end);
	cudaEventSynchronize(end);


	float time=0;
	
	cudaEventElapsedTime(&time,start,end);
	

	int ans[2];

	cudaMemcpy(ans,ad,4,cudaMemcpyDeviceToHost);
	
	cout<<"The minimum element is"<<ans[0]<<endl;
	
	cout<<"The time required dor it is";
	cout<<time<<endl;
	

} 


'The minimum element is1\nThe time required dor it is0.108256\n'